**note that if you run the next cell, you will download 170M!!!**

In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor()])
dataset_train = CIFAR10("./", train=True, download=True, transform=transform)
dataset_test = CIFAR10("./", train=False, download=True, transform=transform)

In [2]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=128, shuffle=True)

# with Linear Layers

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# class Network(nn.Module):
#   def __init__(self, n_input=3 * 32 * 32):
#     super().__init__()
#     self.fc1 = nn.Linear(n_input, 128)
#     self.fc2 = nn.Linear(128, 64)
#     self.fc3 = nn.Linear(64, 32)
#     self.fc4 = nn.Linear(32, 10)

#   def forward(self, x):
#                                   # x.dim -> 128 * 3 * 32 * 32
#     x = x.reshape(x.shape[0], -1) # x.shape[0] = 128
#                                   # x.dim -> 128 * (3 * 32 * 32)
#     x = torch.relu(self.fc1(x))
#     x = torch.relu(self.fc2(x))
#     x = torch.relu(self.fc3(x))
#     x = self.fc4(x)
#     return x

# # model initializing
# model = Network().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

In [5]:
# from sklearn.metrics import f1_score
# import numpy as np

# print("metric is f1_score\n\n", "-"*20)
# for epochs in range(10):
#   model.train()
#   total_loss = 0

#   # Training NN's weights
#   for X_batch, y_batch in train_loader:
#     X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#     optimizer.zero_grad()
#     y_pred = model(X_batch)
#     loss = criterion(y_pred, y_batch)
#     loss.backward()
#     optimizer.step()
#     total_loss += loss.item()

#     # Evaluating NN's weights
#   with torch.no_grad():
#     model.eval()
#     train_f1, test_f1 = [], []

#       # Train
#     for X_batch, y_batch in train_loader:
#       X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#       y_pred = model(X_batch).cpu().numpy().argmax(axis=1)
#       train_f1.append(f1_score(y_pred, y_batch.cpu().numpy(), average="macro"))
#     train_f1_mean = np.mean(train_f1)

#       # Test
#     for X_batch, y_batch in test_loader:
#       X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#       y_pred = model(X_batch).cpu().numpy().argmax(axis=1)
#       test_f1.append(f1_score(y_pred, y_batch.cpu().numpy(), average="macro"))
#     test_f1_mean = np.mean(test_f1)

#     # printing the results of each epochs:
#     print(f"epoch: {epochs+1}, loss = {total_loss / 32:.4f}, train: {np.round(train_f1_mean*100, 4)}%, test: {np.round(test_f1_mean*100, 4)} %","\n", "-"*20)

# with convolution

In [ ]:
class ConvolutionNetwork(nn.Module):
  def __init__(self, n_input=3 * 32 * 32):
    super().__init__()

    # input shape is 128 * 3 * 32 * 32
    self.conv1 = nn.Conv2d(3, 32, (3, 3), (2, 2))   # shape conv1 : 128, 32, 15, 15
    self.conv2 = nn.Conv2d(32, 64, (3, 3), (2, 2))  # shape conv2 : 128, 64, 7, 7
    self.conv3 = nn.Conv2d(64, 256, (3, 3), (2, 2)) # shape conv3 : 128, 256, 3, 3

    self.linear1 = nn.Linear(2304, 128)
    self.linear2 = nn.Linear(128, 10)

  def forward(self, x):
    # Convolutions
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = torch.relu(self.conv3(x))
                                  # X.dim => 128(n_samples) * 256 * 3 * 3
    x = x.reshape(x.shape[0], -1) # X.shape[0] is the n_samples
                                  # x.dim => 128 * 2304
                                  # we made 1152 features of image by conv
    # Linears
    x = torch.relu(self.linear1(x))
    x = torch.relu(self.linear2(x))

    return x

# model initializing
model2 = ConvolutionNetwork().to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.SGD(params=model2.parameters(), lr=0.001)

In [7]:
x = torch.rand(1, 64, 7, 7).to(device)
print(model2.conv3(x).shape)

torch.Size([1, 256, 3, 3])


In [13]:
from sklearn.metrics import f1_score
import numpy as np

print("metric is f1_score\n\n", "-"*20)
for epochs in range(10):
  model2.train()
  total_loss = 0

  # Training NN's weights
  for X_batch, y_batch in train_loader:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    opt.zero_grad()
    y_pred = model2(X_batch)
    loss = criterion(y_pred, y_batch)
    loss.backward()
    opt.step()
    total_loss += loss.item()

    # Evaluating NN's weights
  with torch.no_grad():
    model2.eval()
    train_f1, test_f1 = [], []

      # Train
    for X_batch, y_batch in train_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      y_pred = model2(X_batch).argmax(axis=1).cpu().numpy()
      train_f1.append(f1_score(y_pred, y_batch.cpu().numpy(), average="macro"))
    train_f1_mean = np.mean(train_f1)

      # Test
    for X_batch, y_batch in test_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      y_pred = model2(X_batch).argmax(axis=1).cpu().numpy()
      test_f1.append(f1_score(y_pred, y_batch.cpu().numpy(), average="macro"))
    test_f1_mean = np.mean(test_f1)

    # printing the results of each epochs:
    print(f"epoch: {epochs+1}, loss = {total_loss / 128:.4f}, train: {np.round(train_f1_mean*100, 4)}%, test: {np.round(test_f1_mean*100, 4)} %","\n", "-"*20)

metric is f1_score

 --------------------
epoch: 1, loss = 3.9620, train: 54.2238%, test: 51.8795 % 
 --------------------
epoch: 2, loss = 3.9153, train: 50.8523%, test: 48.275 % 
 --------------------
epoch: 3, loss = 3.8553, train: 55.077%, test: 51.1424 % 
 --------------------
epoch: 4, loss = 3.8106, train: 55.0521%, test: 51.5891 % 
 --------------------
epoch: 5, loss = 3.7483, train: 56.7275%, test: 52.9934 % 
 --------------------
epoch: 6, loss = 3.7043, train: 55.3086%, test: 51.9396 % 
 --------------------
epoch: 7, loss = 3.6609, train: 57.0707%, test: 52.4857 % 
 --------------------
epoch: 8, loss = 3.6054, train: 58.1553%, test: 53.3237 % 
 --------------------
epoch: 9, loss = 3.5645, train: 57.9968%, test: 53.1006 % 
 --------------------
epoch: 10, loss = 3.5136, train: 59.3501%, test: 53.7136 % 
 --------------------
